In [23]:
import requests
import praw
import json
from datetime import datetime
import pytz
import time

# Initialize PRAW with your Reddit API credentials
reddit = praw.Reddit(client_id='PO72CNBqJevVmb8HupBV_Q', 
                     client_secret='kCBrW2Euk37XJMIeb5ZSa6RGA6t1ZQ', 
                     user_agent='Analyse_it')

In [26]:
def word_count(text):
    """Returns the number of words in a given text."""
    return len(text.split())

def fetch_and_analyze_posts(subreddit_name, start_date, end_date):
    """
    Fetches top posts from a subreddit, filters them by a specific date range, and analyzes comments and replies.
    """
    analyzed_posts = []
    total_comments = 0
    total_replies = 0
    total_words = 0
    subreddit = reddit.subreddit(subreddit_name)

    for submission in subreddit.top(time_filter='all', limit=None):
        submission_date = datetime.utcfromtimestamp(submission.created_utc)
        if start_date <= submission_date <= end_date:
            post_data = {
                "thread_id": f"{submission.created_utc}_{submission.id}",  # Unique thread ID
                "title": submission.title,
                "self_text": submission.selftext,
                "score": submission.score,
                "author": submission.author.name if submission.author else None,
                "created_utc": submission.created_utc,
                "created_readable": submission_date.strftime("%d/%m/%Y %H:%M:%S"),
                "comments": []
            }
            post_words = word_count(submission.title) + word_count(submission.selftext)
            total_words += post_words

            submission.comments.replace_more(limit=None)  # Flatten the comment tree
            for comment in submission.comments.list():
                comment_date = datetime.utcfromtimestamp(comment.created_utc)
                total_comments += 1
                comment_words = word_count(comment.body)
                total_words += comment_words
                comment_data = {
                    "body": comment.body,
                    "words": comment_words,
                    "author": comment.author.name if comment.author else None,
                    "created_utc": comment.created_utc,
                    "created_readable": comment_date.strftime("%d/%m/%Y %H:%M:%S"),
                    "replies": []
                }

                for reply in comment.replies:
                    if isinstance(reply, praw.models.Comment):
                        reply_date = datetime.utcfromtimestamp(reply.created_utc)
                        total_replies += 1
                        reply_words = word_count(reply.body)
                        total_words += reply_words
                        reply_data = {
                            "body": reply.body,
                            "words": reply_words,
                            "author": reply.author.name if reply.author else None,
                            "created_utc": reply.created_utc,
                            "created_readable": reply_date.strftime("%d/%m/%Y %H:%M:%S")
                        }
                        comment_data["replies"].append(reply_data)

                post_data["comments"].append(comment_data)

            analyzed_posts.append(post_data)

    return analyzed_posts, total_comments, total_replies, total_words

# Define your date range
start_date = datetime(2023, 9, 22)
end_date = datetime(2024, 1, 31)

# Specify the subreddit
subreddit_name = "iPhone15Pro"

# Fetch, filter posts and analyze data
analyzed_posts, total_comments, total_replies, total_words = fetch_and_analyze_posts(subreddit_name, start_date, end_date)

# Save the analyzed posts and summary to a JSON file
with open("iPhone15pro_Raw_data.json", "w") as file:
    json.dump({
        "posts": analyzed_posts,
        "total_comments": total_comments,
        "total_replies": total_replies,
        "total_words": total_words
    }, file, indent=4, default=str)

print(f"Total analyzed posts: {len(analyzed_posts)}")
print(f"Total comments: {total_comments}")
print(f"Total replies: {total_replies}")
print(f"Total words (posts, comments, replies): {total_words}")
print("Analyzed posts and summary have been saved to analyzed_posts_summary.json.")

Total analyzed posts: 943
Total comments: 57114
Total replies: 28508
Total words (posts, comments, replies): 1981952
Analyzed posts and summary have been saved to analyzed_posts_summary.json.
